In [1]:
# install datasets
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install dask

In [ ]:
# Let's import the library. We typically only need at most two methods:
from datasets import list_datasets, load_dataset

from pprint import pprint

In [ ]:
import dask.dataframe as dd
df = dd.read_parquet('hf://datasets/wikimedia/wikipedia/20231101.ur')

In [ ]:
df.to_csv('a.csv')

['/content/a.csv/0.part',
 '/content/a.csv/1.part',
 '/content/a.csv/2.part',
 '/content/a.csv/3.part']

In [ ]:
import pandas as pd
df2 = pd.read_csv('/content/a.csv/0.part')

In [ ]:
df2

,Unnamed: 0,id,url,title,text
0,0,605,https://ur.wikipedia.org/wiki/%D9%85%D9%88%D8%...,موجودہ سرگرمیاں,\n\nحالات حاضرہ\n\n24 اگست 2006\n\n- جرمنی کی ...
1,1,607,https://ur.wikipedia.org/wiki/%D8%AC%D8%BA%D8%...,جغرافیہ,جغرافیہ ‎(انگریزی: Geography)‎ یونانی زبان کا ...
2,2,608,https://ur.wikipedia.org/wiki/%D9%BE%D8%A7%DA%...,پاکستان,پاکستان (رسمی نام: اسلامی جمہوریۂ پاکستان) ج...
3,3,612,https://ur.wikipedia.org/wiki/%D8%AA%D8%A7%D8%...,تاریخ,تاریخ جامع انسانی کے انفرادی و اجتماعی عمال و ...
4,4,613,https://ur.wikipedia.org/wiki/%D9%84%D8%B3%D8%...,لسانيات,لسانيات ايک ايسا مضمون ہے جس ميں انسانی زبانوں...
...,...,...,...,...,...
70995,70995,335828,https://ur.wikipedia.org/wiki/%D8%A8%D9%86%DB%...,بنی حواء,بنی حواء ( عربی: بني حواء) الجزائر کا ایک الجز...
70996,70996,335829,https://ur.wikipedia.org/wiki/%D8%A8%D9%86%DB%...,بنی راشد، شلف,بنی راشد، شلف ( عربی: بني راشد) الجزائر کا ایک...
70997,70997,335830,https://ur.wikipedia.org/wiki/%D8%A8%D9%88%D9%...,بوقادیر,بوقادیر ( عربی: بوقادير) الجزائر کا ایک الجزائ...
70998,70998,335831,https://ur.wikipedia.org/wiki/%D8%A8%D9%88%D8%...,بوزغایہ,بوزغایہ ( عربی: بوزغاية) الجزائر کا ایک الجزائ...


In [ ]:
! pip install accelerate
! pip install transformers
! pip install einops
! pip install datasets
! pip install peft
! pip install bitsandbytes
from huggingface_hub import notebook_login

notebook_login()
from datasets import load_dataset, concatenate_datasets

datasets_names = [
"medalpaca/medical_meadow_mediqa",
"medalpaca/medical_meadow_mmmlu",
"medalpaca/medical_meadow_medical_flashcards",
"medalpaca/medical_meadow_wikidoc_patient_information",
"medalpaca/medical_meadow_wikidoc",
"medalpaca/medical_meadow_pubmed_causal",
"medalpaca/medical_meadow_medqa",
"medalpaca/medical_meadow_health_advice",
"medalpaca/medical_meadow_cord19",
]

Load and combine datasets
combined_dataset = [load_dataset(name, split="train") for name in datasets_names]
combined_dataset = concatenate_datasets(combined_dataset)
combined_dataset
val_set_size = 0.1

combined_dataset = combined_dataset.train_test_split(test_size=val_set_size,
shuffle=True, seed=2)
combined_dataset
print("question -> ", combined_dataset["train"]["input"][0])
print("answer -> ", combined_dataset["train"]["output"][0])
train_df = combined_dataset["train"].to_pandas()
test_df = combined_dataset["test"].to_pandas()
train_df
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stanford-crfm/BioMedLM", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
def tokenize(sample):
input_ids = tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
return input_ids
from datasets import Dataset

def tokenize_dataset_df(tokenizer, data_df):
data_df["text"] = data_df[["input", "output"]].apply(lambda x: "question: " + x["input"] + " answer: " + x["output"], axis=1)
generated_df = data_df.drop(columns=["input", "output"])
data = Dataset.from_pandas(generated_df)
tokenized_data = data.map(tokenizer, batched=True, desc="Tokenizing data", remove_columns=data.column_names)
return tokenized_data
train_tokenized = tokenize_dataset_df(tokenize, train_df)
test_tokenized = tokenize_dataset_df(tokenize, test_df)
train_tokenized
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.float16
)
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2",
device_map={"":0},
trust_remote_code=True,
quantization_config=bnb_config)

model
from peft import LoraConfig, get_peft_model

config = LoraConfig(r=16,
lora_alpha=16,
target_modules=["Wqkv", "out_proj"],
lora_dropout=0.05,
bias="none",
task_type="CAUSAL_LM")

model = get_peft_model(model, config)
model.print_trainable_parameters()
print(model)
from transformers import TrainingArguments, Trainer

model_output_dir = "Thealth-phi-2-tunned-9_medalpaca_medical_meadow"

training_arguments = TrainingArguments(output_dir=model_output_dir,
per_device_train_batch_size=2,
per_device_eval_batch_size=2,
learning_rate=2e-4,
lr_scheduler_type="cosine",
save_strategy="epoch",
evaluation_strategy='steps',
eval_steps=500,
logging_steps=100,
max_steps=1000,
num_train_epochs=1,
gradient_accumulation_steps=1,
push_to_hub=True)
trainer = Trainer(model=model,
train_dataset=train_tokenized,
eval_dataset=test_tokenized,
args=training_arguments,
data_collator=data_collator)

from peft import PeftModel
from transformer import AutoModelForCausalLModel
import torch
model = AutoModelForCausalLModel.from_pretrained("microsoft/phi-2", trust_remote_code=True, torch_dtype=torch.float32)
peft_model = PeftModel.from_pretrined(model, "TachyHealth/Thealth-phi-2-tunned-9_medalpaca_medical_meadow", from_transformers=True)
model = peft_model.merge_and_unload()
model
model.push_to_hub("TachyHealth/Thealth-phi-2-tunned-9_medalpaca_medical_meadow")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stanford-crfm/BioMedLM", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("TachyHealth/Thealth-phi-2-tunned-9_medalpaca_medical_meadow", trust_remote_code=True, torch_dtype=torch.float32)
inputs = tokenizer(
"""
question: I have 2 apples. My friend gave me another
two apples. I ate 1 apple. Totally how many I have now? answer:
""",
return_tensors="pt",
return_attention_mask=False)
outputs = model.generate(**inputs, max_length=512)
text = tokenizer.batch_decode(outputs)[0]
print(text)

In [ ]:
! pip install accelerate
! pip install transformers
! pip install einops
! pip install datasets
! pip install peft
! pip install bitsandbytes
from huggingface_hub import notebook_login

notebook_login()
from datasets import load_dataset, concatenate_datasets


In [ ]:
from datasets import load_dataset, concatenate_datasets
import threading
import time
import os

datasets_names = [
    "medalpaca/medical_meadow_mediqa",
    "medalpaca/medical_meadow_mmmlu",
    "medalpaca/medical_meadow_medical_flashcards",
    "medalpaca/medical_meadow_wikidoc_patient_information",
    "medalpaca/medical_meadow_wikidoc",
    "medalpaca/medical_meadow_pubmed_causal",
    "medalpaca/medical_meadow_medqa",
    "medalpaca/medical_meadow_health_advice",
    "medalpaca/medical_meadow_cord19",
]

# Function to load a dataset
def load_dataset_function(name):
    try:
        return load_dataset(name, split="train")
    except Exception as e:
        print(f"Failed to load dataset: {name}. Error: {e}")
        return None

# Function to monitor progress (dummy implementation for demonstration purposes)
def monitor_progress(dummy_filepath, timeout):
    start_time = time.time()
    while time.time() - start_time < timeout:
        if os.path.exists(dummy_filepath):
            return True
        time.sleep(1)
    return False

# Function to load a dataset with timeout and progress monitoring
def load_dataset_with_timeout(name, timeout):
    dummy_filepath = "/path/to/dummy/file"  # Replace this with the real path to monitor
    dataset_thread = threading.Thread(target=load_dataset_function, args=(name,))
    dataset_thread.start()

    # Monitor the progress
    if not monitor_progress(dummy_filepath, timeout):
        print(f"No progress detected for dataset: {name}.")
        return None

    dataset_thread.join(timeout)
    if dataset_thread.is_alive():
        print(f"Timeout reached while loading dataset: {name}.")
        return None

    # Assuming the thread finished, get the result
    return load_dataset_function(name)

# Load datasets with a timeout and progress monitoring
loaded_datasets = []
for name in datasets_names:
    dataset = load_dataset_with_timeout(name, timeout=50)  # Timeout in seconds
    if dataset is not None:
        loaded_datasets.append(dataset)

# Combine the successfully loaded datasets
if loaded_datasets:
    combined_dataset = concatenate_datasets(loaded_datasets)
    print(combined_dataset)
else:
    print("No datasets were successfully loaded.")

# Define the validation set size
val_set_size = 0.1

# Check if the combined dataset is not None before splitting
if loaded_datasets:
    # Split the combined dataset into training and validation sets
    combined_dataset = combined_dataset.train_test_split(test_size=val_set_size, shuffle=True, seed=42)
    print(combined_dataset)

In [ ]:
from datasets import load_dataset, concatenate_datasets

datasets_names = [
    "medalpaca/medical_meadow_mediqa",
    "medalpaca/medical_meadow_mmmlu",
    #"medalpaca/medical_meadow_medical_flashcards",
    #"medalpaca/medical_meadow_wikidoc_patient_information",
    "medalpaca/medical_meadow_wikidoc",
    "epfl-llm/guidelines", # newly added from meditron open dataset
    "medalpaca/medical_meadow_medqa",
    #"medalpaca/medical_meadow_health_advice",
    #"medalpaca/medical_meadow_cord19",
]

# Load and combine datasets
combined_dataset = [load_dataset(name, split="train") for name in datasets_names]
combined_dataset = concatenate_datasets(combined_dataset)
combined_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
val_set_size = 0.1

combined_dataset = combined_dataset.train_test_split(test_size=val_set_size,
                                                     shuffle=True, seed=2)
combined_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 808379
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 89820
    })
})

In [ ]:
train_df = combined_dataset["train"].to_pandas()
test_df = combined_dataset["test"].to_pandas()

In [ ]:
import pandas as pd
# Concatenate both DataFrames into one DataFrame
combined_df = pd.concat([train_df, test_df], ignore_index=True)

# Save the combined DataFrame to an Excel file
combined_df.to_excel('huggingface_alpaca_medical.xlsx', index=False)

In [ ]:
combined_df

,instruction,input,output
0,Please summerize the given abstract to a title,SIMPLE SUMMARY: Whilst information on the impa...,Mental Health and Adherence to COVID-19 Protec...
1,Please summerize the given abstract to a title,Coronavirus disease of 2019 (COVID-19) caused ...,Serological survey of antibodies against SARS-...
2,Please summerize the given abstract to a title,BACKGROUND: Available data on clinical present...,Clinical Manifestations and Outcomes of Corona...
3,Please answer with one of the option in the br...,Q:A previously healthy 5-year-old boy is broug...,B: Deferoxamine
4,Please summerize the given abstract to a title,Pandemic measures to mitigate the outbreak of ...,Cancelled routine vaccination appointments due...
...,...,...,...
898194,Please summerize the given abstract to a title,This paper presents a multivariate functional ...,Spatiotemporal Prediction of COVID--19 Mortali...
898195,Please summerize the given abstract to a title,Latin America is a vast heterogeneous territor...,Caregiving for Patients With Frontotemporal De...
898196,Please summerize the given abstract to a title,Myocardial dysfunction is a frequent organ man...,Inotropic Support in the Treatment of Septic M...
898197,Please summerize the given abstract to a title,Sports clubs increasingly are settings for hea...,Building Public Health Capacity through Organi...
